<a href="https://colab.research.google.com/github/kentokura/gob/blob/main/gobgob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#  id
# ( R or B )+ ( a ~ i , z 及びその大文字からなる12文字)
#  id 例 : Rzzzzzzzzzzzz

# choise_piece, board
# [RL, RM, RS, BL, BM, BS, hand] と並んでいる。
# RL ~ BS は[0,0,0,0,0,0,0,0,0] で、[a~i]のマスの場所を表す
# handは[2,2,2,2,2,2]で、RL...BSの順に持ち駒の数を表す

In [3]:
# id , choise_piece, boardの相互変換

# アルファベットを数字に変換 a -> 0
alpha2num = lambda c: ord(c) - ord('a')
# 数字をアルファベットに変換 0 -> a
num2alpha = lambda c: chr(c+64)
# 数字と場所の対応表
num2place_dict = {0:'a', 1:'b', 2:'c', 3:'d', 4:'e', 5:'f', 6:'g', 7:'h', 8:'i'}

# idからchoise_pieceを作成
def id2choise_piece(id):
  choise_piece = [
          [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0]
          ]
  if id.islower() == True:
    return choise_piece
  for i, piece in enumerate(id[1:]):
    pieces_type = i // 2
    # 大文字をchoise_pieseに反映
    if piece.isupper() == True:
      if piece == "Z":
        choise_piece[6][pieces_type] = 1
      else:
        choise_piece[pieces_type][alpha2num(piece.lower())] = 1
      return choise_piece
  return choise_piece

# idからboardを作成
def id2board(id):
  board = [
          [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0]
          ]
  for i, piece in enumerate(id[1:]):
    pieces_type = i // 2
    if piece.isupper() == True:
      continue;
    if piece == "z":
      board[6][pieces_type] += 1
    else: # piece == a ~ i
      board[pieces_type][alpha2num(piece.lower())] = 1
  return board

# color, choise_piece, boardからidへ変換
def create_id(color, choise_piece, board):
  color = color
  choise_piece = choise_piece
  board = board
  pieces = ""
  str = ["zz","zz","zz","zz","zz","zz"]
  # boardをpiecesに反映
  for i, strings in enumerate(board[:6]):
    str[i] = ""
    # piecesを前から更新していく
    # num2place_dictのkeyと[0,0,...,0]の1の位置の数が同じkeyを取得
    for j, p in enumerate(strings):
      if p == 1:
        str[i] += num2place_dict[j]
    str[i] = str[i].ljust(2, "z")
  pieces = ''.join(str)
  # choise_pieceをpiecesに反映
  if choise_piece == [
              [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0]
          ]:
    pass;
  else: 
    # [:6]まで対応
    for i, strings in enumerate(choise_piece[:6]):
      for j, p in enumerate(strings):
        if p == 1:
          char = num2place_dict[j].upper()
          pieces = list(pieces)
          pieces[i*2+1] = char
          pieces = "".join(pieces)
          break;
    # handの対応
    for i, q in enumerate(choise_piece[6]):
      if q == 1:
        char = "Z"
        pieces = list(pieces)
        pieces[i*2+1] = char
        pieces = "".join(pieces)
  id = color + pieces
  return id

In [4]:
class State:
  # 初期化
  def __init__(self, id=None):
    self.id = id if id != None else "rzzzzzzzzzzzz"
    self.mode = "put" if self.id == self.id.lower() else "choise"
    self.pieces = self.id[1:]
    self.color = self.id[0]
    # [RL, RM, RS, BL, BM, BS, hand]
    self.choise_piece = [
            [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
            [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
            [0,0,0,0,0,0]
            ];
    self.board = [
            [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
            [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
            [0,0,0,0,0,0]
            ];
    self.choise_piece = id2choise_piece(self.id)
    self.board = id2board(self.id)
  # 確認
  def __str__(self):
    str = "mode\t= {}\n".format(self.mode)
    str += "pieces\t= {}\n".format(self.pieces)
    str += "color\t= {}\n".format(self.color)
    str += "choise_piece\t= {}\n".format(self.choise_piece)
    str += "board\t= {}\n".format(self.board)
    return str

In [22]:
# 合法手
# actions = [action1, action2,...を合体]
# action = [
#           [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
#           [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
#           [0,0,0,0,0,0]
#           ]  # 1ビットのみ立つ
def legal_actions(state: State) -> list:
  RL = state.board[0]
  RM = state.board[1]
  RS = state.board[2]
  BL = state.board[3]
  BM = state.board[4]
  BS = state.board[5]
  HAND = state.board[6]
  choise_piece_type = 6 # デフォルトはhand
  choise_piece_place = 0
  actions = 0
  if state.mode == "choise": # stateはchoise、ここではchoiseした駒を置く行動を絞り込む
    # 選んだ駒以外行動できませーーーん
    actions = [
               [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
               [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
               [0,0,0,0,0,0]
               ];
    for i in range(6): # handは絶対だめだから見ない
      if state.choise_piece[i] != [0,0,0,0,0,0,0,0,0]:
        choise_piece_type = i
        actions[choise_piece_type] = [1,1,1,1,1,1,1,1,1];
    # choiseしたところに戻すのもだめーー
    for j, c_p in enumerate(state.choise_piece[choise_piece_type]):
      if c_p == 1:
        choise_piece_place = j
        actions[choise_piece_type][choise_piece_place] = 0
    # choiseした駒と同じ大きさ以上があったらおけませーーん
    for i in range(9):
      if RL[i] == 1 or BL[i] == 1: # でかいのがおいてあるよ
        for j in [0,1,2,3,4,5]:
          actions[choise_piece_type][i] = 0 # 比較せず、だめ。
      elif RM[i] == 1 or BM[i] == 1: # 中くらい
        if choise_piece_type not in [0, 3]: # でかならおける、ほかだめ。
          actions[choise_piece_type][i] = 0
      elif RS[i] == 1 or BS[i] == 1: # ちっっちゃい
        if choise_piece_type not in [0,1,3,4]: # でか、ちゅうならおける、ほかだめ。
          actions[choise_piece_type][i] = 0
  elif state.mode == "put": # stateはput、ここではchoiseする駒を選べる行動を絞り込む hand　or 盤面の表面
    # 駒がなければ選べないよ
    actions = state.board
    # handの2を1にしておく
    for i, h in enumerate(HAND):
      if h == 2:
        actions[6][i] = 1
    # rのputなら次の選択はbだね、b以外選べないよ
    if state.color == "r":
      for i in [0,1,2]:
        actions[i] = [0,0,0,0,0,0,0,0,0]
    else: # "b"
      for i in [3,4,5]:
        actions[i] = [0,0,0,0,0,0,0,0,0]
    # 上になんかあったら選べない
    for i in range(9):
      if RL[i] == 1 or BL[i] == 1:
        for j in [1,2,4,5]:
          actions[j][i] = 0
      elif RM[i] == 1 or BM[i] == 1:
        for j in [2,5]:
          actions[j][i] = 0
  return actions

#  test # 
state = State("rabcdefghhzzz")
print("{}モードでテスト\n".format(state.mode))
print("actions = ")
print(legal_actions(state))

putモードでテスト

actions = 
[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 1]]


In [ ]:
state = State("rzzzzzzzzzzzz")
print("{}でテスト".format(state.mode))
print(legal_actions(state))

In [ ]:
state = State("rzzzzzzzzzzzz")
print("{}でテスト".format(state.mode))
print(state.choise_piece)